LangGraph 추천엔진에 맞춘 평가셋ver2 설계

: LangGraph 에이전트가 주어진 제약조건 내에서 얼마나 센스 있게 Reasoning하는지 테스트하는 데 최적화된 평가셋
: 아티스트, 장르는 랜덤 선택/ 온보딩 자료 반영 안된 상태의 평가셋

In [ ]:
import pandas as pd

# V2.1: 입력값(Location, Decibel, Goal) 기반 논리 검증 데이터셋
# 특징: 입력값 간의 상관관계(Conflict)에 집중

data = [
    # ---------------------------------------------------------
    # 1. [기본] 난이도 하 (User Pref 없음 -> 입력값 그대로 매칭)
    # ---------------------------------------------------------
    {
        "ID": 1, "Location": "Library", "Decibel": "Silent", "Goal": "Focus", 
        "User Pref": "None", 
        "Evaluation Criteria": "MUST: Instrumental, White noise, or Classical. BAN: Lyrics, High BPM.",
        "Type": "기본", "Difficulty": "Low"
    },
    {
        "ID": 2, "Location": "Gym", "Decibel": "Loud", "Goal": "Active", 
        "User Pref": "None", 
        "Evaluation Criteria": "MUST: High Energy, EDM, or Upbeat Pop. VIBE: Workout motivation.",
        "Type": "기본", "Difficulty": "Low"
    },
    {
        "ID": 3, "Location": "Home", "Decibel": "Quiet", "Goal": "Sleep", 
        "User Pref": "None", 
        "Evaluation Criteria": "MUST: Ambient, Rain sounds, or Lullaby. BAN: Sudden loud noises, Drums.",
        "Type": "기본", "Difficulty": "Low"
    },

    # ---------------------------------------------------------
    # 2. [취향 반영] 난이도 중 (상황과 취향이 충돌하지 않음)
    # ---------------------------------------------------------
    {
        "ID": 4, "Location": "Cafe", "Decibel": "Moderate", "Goal": "Relax", 
        "User Pref": "Jazz", 
        "Evaluation Criteria": "Genre: Jazz. VIBE: Cozy, Cafe atmosphere, Soft Saxophone/Piano.",
        "Type": "취향반영", "Difficulty": "Mid"
    },
    {
        "ID": 5, "Location": "Park", "Decibel": "Moderate", "Goal": "Active", 
        "User Pref": "K-Pop (NewJeans)", 
        "Evaluation Criteria": "Artist: NewJeans or similar refreshing K-Pop. VIBE: Bright, Driving, Upbeat.",
        "Type": "취향반영", "Difficulty": "Mid"
    },
    {
        "ID": 6, "Location": "Co-working", "Decibel": "Quiet", "Goal": "Focus", 
        "User Pref": "Lo-fi", 
        "Evaluation Criteria": "Genre: Lo-fi/Chillhop. CONDITION: No distracting lyrics (Background friendly).",
        "Type": "취향반영", "Difficulty": "Mid"
    },
    {
        "ID": 7, "Location": "Moving (Car)", "Decibel": "Loud", "Goal": "Neutral", 
        "User Pref": "Pop Hits", 
        "Evaluation Criteria": "Genre: Pop. VIBE: Rhythmic, Sing-along, Driving friendly.",
        "Type": "취향반영", "Difficulty": "Mid"
    },
    {
        "ID": 8, "Location": "Home", "Decibel": "Silent", "Goal": "Consolation", 
        "User Pref": "Ballad (IU)", 
        "Evaluation Criteria": "Artist: IU or Emotional Ballad. VIBE: Sad, Touching, Slow tempo.",
        "Type": "취향반영", "Difficulty": "Mid"
    },

    # ---------------------------------------------------------
    # 3. [충돌 해결] 난이도 상 (입력값 간의 모순 해결 능력 테스트)
    # Logic: Location/Goal의 제약이 User Pref보다 우선순위가 높거나 타협해야 함
    # ---------------------------------------------------------
    {
        "ID": 9, "Location": "Library", "Decibel": "Silent", "Goal": "Focus", 
        "User Pref": "Heavy Metal", 
        "Evaluation Criteria": "STRATEGY: Keep genre vibe but remove noise. ACCEPT: Post-rock, Instrumental Metal. BAN: Screaming.",
        "Type": "충돌해결", "Difficulty": "High"
    },
    {
        "ID": 10, "Location": "Gym", "Decibel": "Loud", "Goal": "Active", 
        "User Pref": "Classical", 
        "Evaluation Criteria": "STRATEGY: Match Energy. ACCEPT: Epic Orchestral, Fast Strings (e.g., Vivaldi Storm). BAN: Slow Adagio.",
        "Type": "충돌해결", "Difficulty": "High"
    },
    {
        "ID": 11, "Location": "Home", "Decibel": "Quiet", "Goal": "Sleep", 
        "User Pref": "Hip-hop", 
        "Evaluation Criteria": "STRATEGY: Remove Aggression. ACCEPT: Lo-fi Hip hop, Jazz-hop, Slow Instrumentals. BAN: Trap, Gangsta Rap.",
        "Type": "충돌해결", "Difficulty": "High"
    },
    {
        "ID": 12, "Location": "Co-working", "Decibel": "Quiet", "Goal": "Focus", 
        "User Pref": "K-Pop (Dance)", 
        "Evaluation Criteria": "STRATEGY: Acoustic Arrangement. ACCEPT: Piano covers, Unplugged ver. BAN: Original dance tracks.",
        "Type": "충돌해결", "Difficulty": "High"
    },
    {
        "ID": 13, "Location": "Cafe", "Decibel": "Moderate", "Goal": "Relax", 
        "User Pref": "EDM", 
        "Evaluation Criteria": "STRATEGY: Downtempo. ACCEPT: Tropical House, Chill House, Lounge. BAN: Festival Big Room.",
        "Type": "충돌해결", "Difficulty": "High"
    },
    {
        "ID": 14, "Location": "Park", "Decibel": "Moderate", "Goal": "Consolation", 
        "User Pref": "Rock", 
        "Evaluation Criteria": "STRATEGY: Soften tone. ACCEPT: Soft Rock, Acoustic Rock, Britpop. BAN: Hard Rock.",
        "Type": "충돌해결", "Difficulty": "High"
    },

    # ---------------------------------------------------------
    # 4. [특수 상황 & 감정] 난이도 상 (Goal의 구체적 해석)
    # ---------------------------------------------------------
    {
        "ID": 15, "Location": "Subway", "Decibel": "Very Loud", "Goal": "Focus", 
        "User Pref": "Classical", 
        "Evaluation Criteria": "STRATEGY: Noise Masking. ACCEPT: Cello/Organ (Low freq), Steady volume. BAN: Dynamic tracks (too quiet parts).",
        "Type": "소음대응", "Difficulty": "High"
    },
    {
        "ID": 16, "Location": "Home", "Decibel": "Moderate", "Goal": "Anger (Vent)", 
        "User Pref": "Rock", 
        "Evaluation Criteria": "GOAL: Catharsis. ACCEPT: Grunge, Punk, Hard Rock. VIBE: Aggressive, High Energy.",
        "Type": "감정분석", "Difficulty": "Mid"
    },
    {
        "ID": 17, "Location": "Home", "Decibel": "Moderate", "Goal": "Anger (Calm)", 
        "User Pref": "Classical", 
        "Evaluation Criteria": "GOAL: Composure. ACCEPT: Dramatic/Majestic Classical. BAN: Irritating high pitches.",
        "Type": "감정분석", "Difficulty": "High"
    },
    {
        "ID": 18, "Location": "Moving", "Decibel": "Loud", "Goal": "Active", 
        "User Pref": "Hip-hop", 
        "Evaluation Criteria": "VIBE: Labor friendly. ACCEPT: Boom Bap, Old School, Rhythmic. VIBE: Steady beat.",
        "Type": "감정분석", "Difficulty": "Mid"
    },
    {
        "ID": 19, "Location": "Cafe", "Decibel": "Moderate", "Goal": "Neutral", 
        "User Pref": "Indie", 
        "Evaluation Criteria": "Genre: Indie Pop/Folk. VIBE: Easy listening, Background music.",
        "Type": "일반", "Difficulty": "Low"
    },
    {
        "ID": 20, "Location": "Library", "Decibel": "Silent", "Goal": "Focus", 
        "User Pref": "Podcast (Talk)", 
        "Evaluation Criteria": "CONFLICT: Talk vs Focus. ACTION: Suggest Instrumental OR Warn about distraction. BAN: Talk shows.",
        "Type": "충돌해결", "Difficulty": "High"
    }
]

# DataFrame 생성
df_v2 = pd.DataFrame(data)

# 실제 LangGraph 입력값에 맞춰 컬럼 순서 정렬
cols = [
    "ID", "Type", "Difficulty", 
    "Location", "Decibel", "Goal", "User Pref",  # 실제 Input 4개
    "Evaluation Criteria"                        # 검증 기준
]
df_v2 = df_v2[cols]

# CSV 파일로 저장
file_name = "evaluation_set_v2_criteria.csv"
df_v2.to_csv(file_name, index=False, encoding="utf-8-sig")

print(f"✅ '{file_name}' 파일 생성 완료!")
print(f"총 {len(df_v2)}개의 케이스가 입력값(Input)과 검증기준(Criteria) 형식으로 저장되었습니다.")

✅ 'evaluation_set_v2_criteria.csv' 파일 생성 완료!
총 20개의 케이스가 입력값(Input)과 검증기준(Criteria) 형식으로 저장되었습니다.
